# <a id='toc1_'></a>[**MICRO-452 Final Report**](#toc0_)
### <a id='toc1_1_1_'></a>[*Group 45: Anton Balykov, Teo Halevi, Cyprien Lacassagne, Seonmin Park*](#toc0_)

# <a id='toc2_'></a>[Table of Contents](#toc0_)


- [Project Objective](#toc3_)    
- [I. Computer Vision](#toc4_)    
  - [I.1 Introduction](#toc4_1_)    
  - [I.2 Camera Set Up](#toc4_2_)    
  - [I.3 Global Obstables and Thymio Detection](#toc4_3_)    
- [II. Global Navigation](#toc5_)    
  - [II.1 Introduction](#toc5_1_)    
  - [II.2 Path Planning](#toc5_2_)    
  - [II.3 Algorithm](#toc5_3_)    
- [III. Local Navigation](#toc6_)    
  - [III.1 Introduction](#toc6_1_)    
  - [III.1 Local Avoidance](#toc6_2_)    
- [IV. Filtering](#toc7_)    
  - [IV.1 Introduction](#toc7_1_)    
  - [IV.2 Methodology](#toc7_2_)    
  - [IV.3 Dynamics of Thymio](#toc7_3_)    
  - [IV.4 Design and Implementation of Filter](#toc7_4_)    
  - [IV.5 Simulation](#toc7_5_)    
- [Conclusion](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc3_'></a>[Project Objective](#toc0_)

This project leverages the Thymio robot to explore and integrate key concepts introduced in the Introduction to Mobile Robotics course. We hereby focus on implementing and demonstrating core components: computer vision, global navigation, motion control, local navigation, and filtering. The mission of the robot is to navigate itself from the designated start position to the specified goal position, successfully avoiding fixed obstacles as well as dynamically placed one. Achieving this requires precise motion control, robust obstable detection, and efficient filtering technique. Such practical application of the five above-mentioned elements would reflect our team's comprehensive understanding of mobile robotics principles.

# <a id='toc4_'></a>[I. Computer Vision](#toc0_)

## <a id='toc4_1_'></a>[I.1 Introduction](#toc0_)

bla bla bla

## <a id='toc4_2_'></a>[I.2 Camera Set Up](#toc0_)

bla bla bla

## <a id='toc4_3_'></a>[I.3 Global Obstables and Thymio Detection](#toc0_)

bla bla bla

.

.

.

.

# <a id='toc5_'></a>[II. Global Navigation](#toc0_)

## <a id='toc5_1_'></a>[II.1 Introduction](#toc0_)

One main component of this project is navigation. Here, we discuss about our implementation of the global navigation. The main objectives of the global navigation is to find a collision-free path from a source and a goal, on a map filled with polygons acting as obtacles. The robot has to reach the goal while avoiding the obstacles. The strategy relies on a metric map and visibility graph.

## <a id='toc5_2_'></a>[II.2 Path Planning](#toc0_)

The class `Navigation` is constructed by receiving the position of the robot (source), the goal, and the corners of each obstacle. The latter are detected using polygon approximation in the class `ComputerVision`. For each obstacle, they are grouped into a list of points, representing its respective vertices.<br>

However, before going into graph creation, the polygons need to be expanded in order to account for the size of the robot. Otherwise, the robot would be considered as a single point object and bump into the obstacles.

Then, a visibility graph is built using the open-source library `pyvisgraph`.
Finally, the path is computed using the function `graph.get_shortest_path`, and returned to the main function to assess the robot's deviation from the goal.

## <a id='toc5_3_'></a>[II.3 Algorithm](#toc0_)

The obstacles are expanded in an intuitive manner: for each vertex, a point outside of the polygon is computed based on the direction of the two adjacent edges. The distance of the expansion is simply equals to the radius (diagonal) of the robot.<br>

The graph is constructed using a basic graph algorithm. Then calculations are made to filter out the connections violating free-space movement, i.e. connections between two vertices going through one or more obstacles.<br>
This yields a visibility graph connecting nodes that will form candidate paths from source to goal.
Then Dijkstra's algorithm is then used to find the shortest path in the graph.


# <a id='toc6_'></a>[III. Motion Control & Local Navigation](#toc0_)

## <a id='toc6_1_'></a>[III.1 Introduction](#toc0_)

After extracting the information from the camera and defining the objective for the robot to go to, the next step is to apply motion control and local navigation to guide the robot to the goal. 


## <a id='toc6_2_'></a>[III.2 Thymio Class](#toc0_)

To read the readings from Thymio's sensors and control the robot's movement, the `Thymio` class was implemented.

Main tasks of this class are:
- Communicate with the robot via `ClientAsync` from [`tdmclient`](https://pypi.org/project/tdmclient/) library
- Read the sensor values (horizontal proximity and wheels speed)
- Control the robot's movement (set the speed of the wheels)

The class consists of multiple constants and functions to achieve the above tasks.

---
### Constants
1. Constants to store strings of the sensor names and motor names:
    ```
        SENSORS_HORIZONTAL = "prox.horizontal"
        SPEED_LEFT = "motor.left.speed"
        SPEED_RIGHT = "motor.right.speed"
        
        MOTOR_LEFT = "motor.left.target"
        MOTOR_RIGHT = "motor.right.target"
    ```

2. Constants to set the sensitivity of obstacle detection, reaching the goal and when to start turning towards the next target:
    ```
        GOAL_THRESHOLD = 10
        OBSTACLE_THRESHOLD = 1800
        ANGLE_THRESHOLD = 0.01
    ```

3. `W` matrix with the weights for the horizontal sensors during local navigation:
    ```
        SCALE = 0.03
        W = np.array([[2, 1, -4, -1, -2], [-2, -1, -2, 1, 2]]) * SCALE
    ```
    
    The visualization of the weights is shown below (**from the Week 3 exercise**):
       ![Weights visualization](./report-images/ANN_robot_control.png)   

    Scaling factor is used to adjust the target speed of the robot.

4. Speed of the robot:
    ```
        SPEED = 70
    ```
    Used to set the speed of the robot's wheels. Was chosen so the robot moves at a reasonable speed enough to turn and not too slow.

5. A constant to account for a sharp turn:
    ```
        SHARP_TURN_THRESHOLD = np.pi / 6
    ```
    As the movement controller was chosen to move in straight lines or turn if there is an angle between the robot's orientation and the target, sometimes the angle and the radius of turn can be too big to reach the next target. THis constant is used in the movement controller to let robot turn in-place instead of more smooth turns. Set to 30 degrees so that robot moves more in straight lines and truns faster.
   
---
### Functions
1. The instance of the class is created in the `main.py` file to connect to the robot.
    ```
        def __init__(self):
            self.client = ClientAsync()
            self.node = aw(self.client.wait_for_node())
            aw(self.node.lock())
        
            self.goal = None
            self.position = None
            self.orientation = None
            self.is_on_goal = False
            self.is_kidnapped = False
    ```
    This function also sets the flags needed to define the state of the robot during the simulation.

2. The function to set the motor speeds of the Thymio (used when applying any movement to the robot):
    ```
        def set_motors(self, left_motor: int, right_motor: int, verbose: bool = False):
            if verbose:
                print("\t\t Setting speed : ", left_motor, right_motor)
        
            aw(self.node.set_variables({
                self.MOTOR_LEFT: [left_motor],
                self.MOTOR_RIGHT: [right_motor]
            }))
    ```

3. Functions to read sensor data (used to detect obstacles for local navigation and to estimate the robot's position for filtering):
    ```
        def get_horizontal_sensors(self, verbose: bool = False):
            aw(self.client.wait_for_status(self.client.NODE_STATUS_READY))
            aw(self.node.wait_for_variables({self.SENSORS_HORIZONTAL}))
    
            values = self.node.var[self.SENSORS_HORIZONTAL]
    
            if verbose:
                print("\t\t Sensor values : ", values)
    
            return np.array(values)[:5]
    
        def get_wheels_speed(self, verbose: bool = False):
            aw(self.client.wait_for_status(self.client.NODE_STATUS_READY))
            aw(self.node.wait_for_variables({self.SPEED_LEFT, self.SPEED_RIGHT}))
    
            left_speed = self.node.var[self.SPEED_LEFT]
            right_speed = self.node.var[self.SPEED_RIGHT]
    
            if verbose:
                print("\t\t Wheel speeds : ", left_speed, right_speed)
    
            return np.array([left_speed[0], right_speed[0]])
    ```

4. Setters of the position, orientation and goal of the robot:
    ```
        def set_goal(self, goal):
            self.goal = goal
    
        def set_position(self, position: np.ndarray):
            self.position = position
    
        def set_orientation(self, orientation):
            self.orientation = orientation
    ```

5. Functions to stop and correctly delete the instance of the class:
    ```
        def stop(self):
            self.set_motors(0, 0)
    
        def __del__(self):
            aw(self.node.unlock())
    ```

6. Functions to account for kidnapping:
    ```
        def kidnap(self):
            self.is_kidnapped = True
            self.stop()
    
        def recover(self):
            self.is_kidnapped = False
    ```

7. Function to apply the motion control to the robot:
    ```
        def move_to_point(self, target: np.ndarray, verbose: bool = False):
            path = target - self.position

            dst = np.sqrt(np.sum(np.square(path)))
    
            if dst <= self.GOAL_THRESHOLD:
                if np.sqrt(np.sum(np.square(self.position - self.goal))) <= self.GOAL_THRESHOLD:
                    self.is_on_goal = True
    
                return True
    
            angle = self.orientation - np.arctan2(path[1], path[0])
    
            if angle > np.pi:
                angle = angle - 2 * np.pi
    
            if verbose:
                print("Distance: ", dst)
                print("Orientation: ", self.orientation)
                print("Angle: ", angle)
                print("Position: ", self.position)
                print("Target: ", target)
    
            if angle > self.ANGLE_THRESHOLD:
                if angle > self.SHARP_TURN_THRESHOLD:
                    self.set_motors(self.SPEED, -self.SPEED)
                    time.sleep(0.5)
                else:
                    self.set_motors(self.SPEED, -self.SPEED)
            elif angle < -self.ANGLE_THRESHOLD:
                if angle < -self.SHARP_TURN_THRESHOLD:
                    self.set_motors(-self.SPEED, self.SPEED)
                    time.sleep(0.5)
                else:
                    self.set_motors(-self.SPEED, self.SPEED)
    
            self.set_motors(self.SPEED, self.SPEED)
    
            return False
    ```
    The `if dst <= self.GOAL_THRESHOLD` check is made in order to check if the target has been reached. There is an additional check for the goal to be reached in case the robot is already on the goal (in that case the `is_on_goal` flag is set to `True`).

    Here angle is computed as the difference between the orientation of the robot and the angle of the path to the target. If the angle is greater than the threshold, the robot turns towards the target. Otherwise, it moves forward.
    Additional check is made in order to have angles within the range of $[-\pi, \pi]$.
    In case of sharp turn, that is defined by the constant `SHARP_TURN_THRESHOLD` (described above), the robot turns in-place (done with the `time.sleep(0.5)` to make sure the robot has turned enough before moving forward).

## <a id='toc6_2_'></a>[III.3 Global Navigation Movement](#toc0_)

Global navigation is implemented via following the points on the global path defined by the path-planning algorithm explained in the previous section.

```
    # MOVEMENT
    target = global_path[0]
    if thymio.move_to_point(np.array([target.x * -1, target.y])):
        print(f"Reached checkpoint {check_num}")
        check_num += 1
        global_path.pop(0)
```

Target is chosen as the next point on the global path. The robot moves towards the target point until it reaches it. Once the target is reached, the point is removed from the global path and the robot moves towards the next point.

*The `target.x * -1` is used to flip the $x$-axis of the target point as the coordinate system of the camera is inverted (the `(0,0)` point is in the top-left corner and the $x$-axis looks to the left compared to the $y$-axis).

![Coordinate system of the camera](./report-images/coordinate_system.jpg)

The inversion was done in order to have the standard Decart coordinate system and to not confuse the robot's movement.

## <a id='toc6_2_'></a>[III.4 Local Navigation Movement](#toc0_)

For the local navigation part the obstacle avoidance algorithm similar to the exercise of **Week 3** is implemented:
```
    def local_navigation(self):
         while np.any(self.get_horizontal_sensors() > self.OBSTACLE_THRESHOLD):
            motor_values = self.W @ self.get_horizontal_sensors().T
            left_motor = int(motor_values[0])
            right_motor = int(motor_values[1])

            self.set_motors(left_motor, right_motor)

        self.set_motors(self.SPEED, self.SPEED)
        time.sleep(1)
```

As long as the horizontal sensors of the robot detect an obstacle, the previously described `W` matrix of weights is multiplied by the sensor values and added to the speed of the robot. This way the robot is able to avoid the obstacles by driving around them. The `time.sleep(1)` is used to make sure that it takes some time for the robot to move around the obstacle and only after that continue with the global navigation.

The algorithm is similar to the one mentioned in **Week 3 exercise**:
![Local Navigation algorithm](./report-images/ANN_algorithm.png)

The local navigation routine is handled in the main loop of the program:
```
    # LOCAL NAVIGATION
    if not thymio.is_kidnapped and np.any(thymio.get_horizontal_sensors() > thymio.OBSTACLE_THRESHOLD):
        thymio.local_navigation()
```

# <a id='toc7_'></a>[IV. Filtering](#toc0_)

## <a id='toc7_1_'></a>[IV.1 Introduction](#toc0_)


There is a need for a method that compensates the defect that there is no direct mapping between Thymio's sensors and the values which are used for control. The absolute position of the robot can be obtained directly from the camera and global path planning techniques, but the robot may have low precision and update rate, causing drifts when the velocity or acceleration is integrated. We can address this challenge by implementing Kalman filter as it provides a sound framework to use all available measurements in an optimal way. It relies on a model for the system structure and the effect of disturbances both on the state and on the measurements.

The Kalman filter we implemented estimates the position and speed of the robot when it moves along the planned path. Specifically, we implemented extended Kalman filter to take into account the system involves nonlinear dynamics that would need to be linearized around the current estimate, focusing on local optimality.


## <a id='toc7_2_'></a>[IV.2 Methodology](#toc0_)

The Thymio robot moves at constant speed across a pattern made of black and white stripes of 50 mm, identical to one of the activities found in Exercise 8 of the course. The actual mean velocity of the two wheels and the mean reflected light read by the two ground sensors are sent in an event at a constant rate. The python code receives these values, updates the estimated velocity and position of the robot, and displays them continuously in a plot.


To obtain velocity parameters, we began with estimating Thymio's speed by let it cross seven blocks that are 50 mm each with a sample rate of 0.1 seconds and marked peaks of the ground sensor meaasuremnet indicating that it has crossed evey block. With known and obtained information, we could obtain the conversion factor of 0.4 to conver Thymio's speed to mm/s.

![Find Peaks](find_peaks.png "Fine Peaks")

Now, to obtain speed variance, we observed the data where the robot was moving by plotting the robot's speed and computed average velocity and thus the speed variance by diving the average speed with the robot's speed in mm/s.

![Measured Vel](measured_vel.png "Measured Vel")

With the assumption that half of the variance is cause by the measurement and the other half is caused by the perturbation to the states, we obtain 

$q_v = r_v = 16.08621$ $mm^2/s^2$ $\space$ where $\space$ $\sigma^2_{vel} = 32.17242$ $mm^2/s^2$

Variances on position state and measurement as well as those for the angle for orientation ($\sigma^2_{pos}$ and $\sigma^2_{\theta}$) were an arbitrary value that were tuned manually for the specific Thymio robot that is going to be used for demonstration.

## <a id='toc7_3_'></a>[IV.3 Dynamics of Thymio](#toc0_)

To begin with the implementation, the state space model is

$\hat{x}_{k-1|k-1} =  \begin{bmatrix}
                        x_{pos_{k-1}} \\
                        y_{pos_{k-1}} \\
                        \theta_{k-1} \\
                        v_{left_{k-1}} \\
                        v_{right_{k-1}} \\
                        \end{bmatrix}$

Under control input vector,

$ u =[v_{trans}, v_{rot}] $, $\space$ where $\space$ $v_{tran} = \frac{v_{left} + v_{right}}{2} $ $\space$ and $\space$ $ v_{rot} = \frac{v_{left} - v_{right}}{l_{base}} $

<img src="Thymio_odometry.png" width="571"/>

Then, we can see that the robot's new position and orientation are predicted in the following way

$x_{pos_{k}} = x_{pos_{k-1}} + v_{trans_{k}} \cdot \cos(\theta_{k-1}) \cdot dk$

$y_{pos_{k}} = y_{pos_{k-1}} + v_{trans_{k}} \cdot \sin(\theta_{k-1}) \cdot dk$

$\theta_{k} = \theta_{k-1} + v_{rot_{k}} \cdot dk$

## <a id='toc7_4_'></a>[IV.4 Design and Implementation of Filter](#toc0_)



Before diving into the steps of the extended Kalman filter we implemented, we would like to disclaim that the notations and formulas were referenced from lecture slides 8 of the Mobile Robotics course and online tutorial at https://automaticaddison.com/extended-kalman-filter-ekf-with-python-code-example/.

We have introduced that the extended Kalman filter takes into account nonlinear system dynamics in the previous sections. In order to make prediction of the next state, We use a nonlinear motion model $f(x, u)$. That is, we can formulate their relation as:

$\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_k)$

Our $f(x, u)$ is then introduced as

$f(\hat{x}_{k-1|k-1}, u_k) = \begin{bmatrix}
1 & 0 & 0 & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & 0 & -0.5 \cdot \sin(\theta_{k-1}) \cdot dk & -0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -\frac{dk}{2 \cdot l_{\text{base}}} & \frac{dk}{2 \cdot l_{\text{base}}} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

In this prediction step of the filter, we make a use of the predicted covariance matrix $P_{k-1|k-1}$.

$P_{k-1|k-1} = \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

as each column corresponds to the state variables, from the first column to the last column, being $x_{pos_{k-1}}$, $y_{pos_{k_1}}$, $\theta_{k-1} $, $v_{left_{k-1}}$, and $v_{right_{k-1}}$, respectively.

The updated (current) covariance matrix is as follows:

$P_{k|k-1} = F_{k}P_{k-1|k-1}F_{k}^{T} + Q_{k}$

where $F_{k}$ is the Jacobian of $f_{k}$ matrix, and $Q_{k}$ is the noise covariance matrix of the state model. The $Q_{k}$ matrix specifically represents the degree or the extend of how much the actual motion of Thymio deviates from its assumed state space model which would be the path that it should follow for our case.

We can write the two matrices as below:

$F_{k|k-1} = 
\begin{bmatrix}
1 & 0 & v_{trans} \cdot \sin(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & v_{trans} \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -\frac{dk}{2 \cdot l_{base}} & \frac{dk}{2 \cdot l_{base}} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

$Q_{k|k-1} = \alpha \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

where $\alpha$ is a scaling factor, which we have initialized as $\alpha =10$, such that 

$Q_{k-1|k-1} = 10 \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

We set $\alpha$ large enough so that the filter tracks large changes in the wheel sensor measurements more closely as if it was smaller.

In the updating process of the extended Kalman filter, we make a use of the term *innovation* which is the difference between the actual measurements and predicted observations. Let $z_{k}$ be the observation vector and $i_{k}$ be the innovation term. Then, we can compute:

$i_{k} = z_{k} - H_{k}\hat{x}_{k-1}$ 

where $H_{k}$ is the measurement matrix that is used to convert the predicted state estimate at time $k$ into predicted sensor measurements at time $k$.

In this project, we have two cases that can happen. One of them is when Thymio gets information from the camera, we name it as "when it has vision," and the other is when it does not get information from the camera, "when it has no vision," specifically when the camera is covered.

When it has vision, 

$H_{k} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

When it does not have vision, 

$H_{k} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

as the robot will have to rely only on its wheel sensor measurements in this scenario.

With the innovation $y_{k}$, we obtain the innovation covariance matrix $S_{k}$ to be used for the computation of Kalman Gain $K_{k}$.

$S_{k} = H_{k}P_{k|k-1}H_{k}^{T} + R_{k}$

where $R_{k}$ is the sensor measurement noise covariance matrix. It also has different dimensionality depending on if Thymio has vision or not.

When it has vision,

$R_{k} = \beta \cdot \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

When it does not have vision,

$R_{k} = \gamma \cdot \begin{bmatrix}
\sigma^2_{vel} & 0\\ 
0 & \sigma^2_{vel} 
\end{bmatrix}$

with the same logic of $H_{k}$ matrices, and constants $\beta = 1$ and $\gamma = 1$ were defined through trial and error experiments just like $\alpha$ of the $Q_{k}$ matrix.

The Kalman Gain matrix is computed as follows:

$K_{k} = P_{k|k-1}H_{k}^{T}S_{k}^{-1}$

and we can update our estimated state and its covariance matrix by

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_{k}i_{k}$ $\space$ and $\space$ $ P_{k|k} = (I - K_{k}H_{k})P_{k|k-1}$ $\space$,  respectively.


The above steps are implemented as a class ***KalmanFilterExtended*** and the class is called in the **main.py** file where all the five components of the project are integrated.

The function ***run_ekf()*** ensures that the extened filter is intiated and the snippet from **main.py** below ensures that the filter is running in a loop throughout the entire mission of the robot.

Additionally, we have set parameters that defines *kidnapping* in terms of the difference in the Thymio's orientation and position exceeding the threshold values. Specifically, we have set the threshold for orientation and position to be 35cm and 60°, and exceeding either (or both) of these values would be considered getting kidnaped.

**Codes to be displayed here **

## <a id='toc7_5_'></a>[IV.5 Simulation](#toc0_)

to be shown later

# <a id='toc8_'></a>[Conclusion](#toc0_)
It has been an incredible journey to embark on this project within the constrained timeline of just three weeks, replete with challenges and invaluable learning opportunities. From the very beginning, we have had to face numerous obstacles, such as technical errors and conceptual hurdles, which have pushed us to refine our problem-solving skills and deepen our understanding of mobile robotics. Despite these challenges, our team has been able to overcome setbacks and achieve results we are proud of through persistence, discipline, and a collaborative spirit.

Working through the complexities of the project was as rewarding as it was demanding. From understanding the theoretical basis to the implementation of practical solutions, each step provided an opportunity to reinforce and showcase our grasp of the principles of mobile robotics. The hands-on nature of the project tested not only our knowledge but also helped us grow as engineers, fostering resilience, adaptability, and creativity in tackling real-world problems.

We would like to express our deep gratitude to everyone who guided and supported us through this journey. We would like to thank Professor Mondada, who inspired us with his experience and vision, and the teaching assistants, who were always ready to help, explain doubts, and give us valuable insights. Their encouragement and mentorship played a very important role in driving the success of our project.

It is not only an academic milestone but also a source of personal and professional growth, and we will be carrying the lessons learned herein as we progress in our future endeavors related to robotics and beyond.